In [5]:
import copy
import glob
import os
import time

import gym
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

from envs import make_env 

In [6]:
class args:
    def __init__(self):
        self.env_name='PongNoFrameskip-v4'
        self.seed=1
        self.log_dir=''
        self.save_dir='saved_models'
        self.cuda=False
        self.num_stack=4
        self.num_steps=5
        self.num_processes=16
        self.lr=7e-4
        self.eps=1e-5
        self.alpha=.99
        self.max_grad_norm=.5
        self.value_loss_coef=.5
        self.entropy_coef=.1
        self.num_frames=8e6
        self.use_gae=False
        self.gamma=.99
        self.tau=.95
        self.save_interval=1000
        self.log_interval=100
        self.vis_interval=100
        self.load_model=False
        self.save_model=True
        
args = args()

SAVE_PATH = "saved_models/a2c_121717.pt"
LOAD_PATH = "saved_models/a2c_121717.pt"

In [7]:
num_updates = int(args.num_frames) // args.num_steps // args.num_processes

torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

In [12]:
def main():
    os.environ['OMP_NUM_THREADS'] = '1'

    envs = [make_env(args.env_name, args.seed, i, args.log_dir)
                for i in range(args.num_processes)]

    if args.num_processes > 1:
        envs = SubprocVecEnv(envs)
    else:
        envs = DummyVecEnv(envs)

    obs_shape = envs.observation_space.shape
    obs_shape = (obs_shape[0] * args.num_stack, *obs_shape[1:])

    global actor_critic
    
    actor_critic = CNNPolicy(obs_shape[0], envs.action_space)
  
    if args.load_model:
        actor_critic.load_state_dict(torch.load(LOAD_PATH))

    action_shape = 1

    if args.cuda:
        actor_critic.cuda()

    optimizer = optim.RMSprop(actor_critic.parameters(), args.lr, eps=args.eps, alpha=args.alpha)

    rollouts = RolloutStorage(args.num_steps, args.num_processes, obs_shape, envs.action_space,\
                              actor_critic.state_size)
    
    current_obs = torch.zeros(args.num_processes, *obs_shape)

    def update_current_obs(obs):
        shape_dim0 = envs.observation_space.shape[0]
        obs = torch.from_numpy(obs).float()
        if args.num_stack > 1:
            current_obs[:, :-shape_dim0] = current_obs[:, shape_dim0:]
        current_obs[:, -shape_dim0:] = obs

    obs = envs.reset()
    update_current_obs(obs)

    rollouts.observations[0].copy_(current_obs)

    # These variables are used to compute average rewards for all processes.
    episode_rewards = torch.zeros([args.num_processes, 1])
    final_rewards = torch.zeros([args.num_processes, 1])

    if args.cuda:
        current_obs = current_obs.cuda()
        rollouts.cuda()

    start = time.time()
    for j in range(num_updates):
        for step in range(args.num_steps):
            # Sample actions
            value, action, action_log_prob, states = actor_critic.act(Variable(rollouts.observations[step], volatile=True),
                                                                      Variable(rollouts.states[step], volatile=True),
                                                                      Variable(rollouts.masks[step], volatile=True))
            cpu_actions = action.data.squeeze(1).cpu().numpy()

            # Obser reward and next obs
            obs, reward, done, info = envs.step(cpu_actions)
            reward = torch.from_numpy(np.expand_dims(np.stack(reward), 1)).float()
            episode_rewards += reward

            # If done then clean the history of observations.
            masks = torch.FloatTensor([[0.0] if done_ else [1.0] for done_ in done])
            final_rewards *= masks
            final_rewards += (1 - masks) * episode_rewards
            episode_rewards *= masks

            if args.cuda:
                masks = masks.cuda()

            if current_obs.dim() == 4:
                current_obs *= masks.unsqueeze(2).unsqueeze(2)
            else:
                current_obs *= masks

            update_current_obs(obs)
            rollouts.insert(step, current_obs, states.data, action.data, action_log_prob.data, value.data, reward, masks)

        next_value = actor_critic(Variable(rollouts.observations[-1], volatile=True),
                                  Variable(rollouts.states[-1], volatile=True),
                                  Variable(rollouts.masks[-1], volatile=True))[0].data

        rollouts.compute_returns(next_value, args.use_gae, args.gamma, args.tau)

        values, action_log_probs, dist_entropy, states = actor_critic.evaluate_actions(Variable(rollouts.observations[:-1].view(-1, *obs_shape)),
                                                                                       Variable(rollouts.states[0].view(-1, actor_critic.state_size)),
                                                                                       Variable(rollouts.masks[:-1].view(-1, 1)),
                                                                                       Variable(rollouts.actions.view(-1, action_shape)))

        values = values.view(args.num_steps, args.num_processes, 1)
        action_log_probs = action_log_probs.view(args.num_steps, args.num_processes, 1)

        advantages = Variable(rollouts.returns[:-1]) - values
        value_loss = advantages.pow(2).mean()

        action_loss = -(Variable(advantages.data) * action_log_probs).mean()

        optimizer.zero_grad()
        (value_loss * args.value_loss_coef + action_loss - dist_entropy * args.entropy_coef).backward()

        nn.utils.clip_grad_norm(actor_critic.parameters(), args.max_grad_norm)

        optimizer.step()
 
        rollouts.after_update()

        if j % args.save_interval == 0 and args.save_model:
            # A really ugly way to save a model to CPU
            save_model = actor_critic
            if args.cuda:
                save_model = copy.deepcopy(actor_critic).cpu()
                
            torch.save(actor_critic.state_dict(), SAVE_PATH)

        if j % args.log_interval == 0:
            end = time.time()
            total_num_steps = (j + 1) * args.num_processes * args.num_steps
            print("Updates {}, num timesteps {}, FPS {}, mean/median reward {:.1f}/{:.1f}, min/max reward {:.1f}/{:.1f}, entropy {:.5f}, value loss {:.5f}, policy loss {:.5f}".
                format(j, total_num_steps,
                       int(total_num_steps / (end - start)),
                       final_rewards.mean(),
                       final_rewards.median(),
                       final_rewards.min(),
                       final_rewards.max(), dist_entropy.data[0],
                       value_loss.data[0], action_loss.data[0]))
            print("\n rollout returns", rollouts.returns, "\nvalues", values)
            
main()

Updates 0, num timesteps 80, FPS 2, mean/median reward 0.0/0.0, min/max reward 0.0/0.0, entropy 1.58978, value loss 0.04211, policy loss 0.22921

 rollout returns 
(0 ,.,.) = 
  0.3817
  0.3835
  0.3883
  0.4000
  0.3911
  0.3947
  0.3945
  0.3849
  0.3966
  0.3997
  0.3972
  0.3838
  0.3983
  0.3861
  0.3827
  0.3892

(1 ,.,.) = 
  0.3856
  0.3874
  0.3923
  0.4041
  0.3951
  0.3987
  0.3985
  0.3888
  0.4007
  0.4037
  0.4012
  0.3877
  0.4024
  0.3900
  0.3866
  0.3931

(2 ,.,.) = 
  0.3895
  0.3913
  0.3962
  0.4082
  0.3991
  0.4027
  0.4025
  0.3927
  0.4047
  0.4078
  0.4052
  0.3916
  0.4064
  0.3939
  0.3905
  0.3971

(3 ,.,.) = 
  0.3934
  0.3953
  0.4002
  0.4123
  0.4031
  0.4068
  0.4066
  0.3967
  0.4088
  0.4119
  0.4093
  0.3956
  0.4105
  0.3979
  0.3945
  0.4011

(4 ,.,.) = 
  0.3974
  0.3992
  0.4043
  0.4164
  0.4072
  0.4109
  0.4107
  0.4007
  0.4129
  0.4161
  0.4135
  0.3996
  0.4147
  0.4019
  0.3984
  0.4051

(5 ,.,.) = 
  0.4014
  0.4033
  0.4084
  0.4206
  0

Updates 500, num timesteps 40080, FPS 127, mean/median reward -20.2/-21.0, min/max reward -21.0/-17.0, entropy 1.78284, value loss 0.06730, policy loss -0.02376

 rollout returns 
(0 ,.,.) = 
 -1.8078
 -1.8041
 -1.8135
 -1.8125
 -2.8046
 -1.8128
 -1.8113
 -1.8026
 -2.7641
 -1.7981
 -1.8140
 -1.8122
 -1.8079
 -1.8060
 -1.8159
 -1.8135

(1 ,.,.) = 
 -1.8261
 -1.8223
 -1.8319
 -1.8308
 -1.8228
 -1.8311
 -1.8296
 -1.8208
 -2.7921
 -1.8163
 -1.8323
 -1.8305
 -1.8262
 -1.8242
 -1.8342
 -1.8318

(2 ,.,.) = 
 -1.8445
 -1.8408
 -1.8504
 -1.8493
 -1.8413
 -1.8496
 -1.8480
 -1.8392
 -2.8203
 -1.8346
 -1.8508
 -1.8490
 -1.8446
 -1.8426
 -1.8527
 -1.8503

(3 ,.,.) = 
 -1.8632
 -1.8593
 -1.8691
 -1.8680
 -1.8599
 -1.8683
 -1.8667
 -1.8578
 -2.8488
 -1.8531
 -1.8695
 -1.8677
 -1.8632
 -1.8612
 -1.8714
 -1.8690

(4 ,.,.) = 
 -1.8820
 -1.8781
 -1.8879
 -1.8868
 -1.8786
 -1.8871
 -1.8856
 -1.8766
 -2.8775
 -1.8719
 -1.8884
 -1.8866
 -1.8821
 -1.8800
 -1.8903
 -1.8879

(5 ,.,.) = 
 -1.9010
 -1.8971
 -1.9

Updates 1000, num timesteps 80080, FPS 149, mean/median reward -20.2/-20.0, min/max reward -21.0/-19.0, entropy 1.78848, value loss 0.04575, policy loss -0.00901

 rollout returns 
(0 ,.,.) = 
 -2.3868
 -1.4124
 -1.4154
 -1.4187
 -1.4170
 -1.4113
 -1.4111
 -1.4110
 -1.4162
 -1.4153
 -1.4094
 -1.4150
 -1.4127
 -1.4087
 -1.4168
 -1.4180

(1 ,.,.) = 
 -2.4110
 -1.4267
 -1.4297
 -1.4330
 -1.4313
 -1.4256
 -1.4253
 -1.4253
 -1.4305
 -1.4296
 -1.4236
 -1.4293
 -1.4270
 -1.4229
 -1.4311
 -1.4323

(2 ,.,.) = 
 -2.4353
 -1.4411
 -1.4441
 -1.4475
 -1.4457
 -1.4400
 -1.4397
 -1.4397
 -1.4450
 -1.4441
 -1.4380
 -1.4437
 -1.4414
 -1.4373
 -1.4456
 -1.4468

(3 ,.,.) = 
 -2.4599
 -1.4556
 -1.4587
 -1.4621
 -1.4603
 -1.4545
 -1.4543
 -1.4542
 -1.4596
 -1.4586
 -1.4525
 -1.4583
 -1.4560
 -1.4518
 -1.4602
 -1.4614

(4 ,.,.) = 
 -1.4747
 -1.4703
 -1.4735
 -1.4769
 -1.4751
 -1.4692
 -1.4690
 -1.4689
 -1.4743
 -1.4734
 -1.4672
 -1.4730
 -1.4707
 -1.4665
 -1.4749
 -1.4762

(5 ,.,.) = 
 -1.4896
 -1.4852
 -1.

Updates 1500, num timesteps 120080, FPS 142, mean/median reward -20.4/-21.0, min/max reward -21.0/-19.0, entropy 1.79118, value loss 0.04904, policy loss -0.02508

 rollout returns 
(0 ,.,.) = 
 -1.3451
 -1.3215
 -1.4823
 -1.5386
 -2.5408
 -1.3240
 -1.5751
 -1.3233
 -1.4458
 -1.5543
 -2.2874
 -1.6266
 -1.5870
 -1.2959
 -1.6165
 -1.2742

(1 ,.,.) = 
 -1.3587
 -1.3349
 -1.4973
 -1.5541
 -1.5563
 -1.3374
 -1.5910
 -1.3367
 -1.4604
 -1.5700
 -2.3105
 -1.6430
 -1.6030
 -1.3090
 -1.6328
 -1.2871

(2 ,.,.) = 
 -1.3724
 -1.3484
 -1.5124
 -1.5698
 -1.5721
 -1.3509
 -1.6071
 -1.3502
 -1.4752
 -1.5859
 -2.3338
 -1.6596
 -1.6192
 -1.3222
 -1.6493
 -1.3001

(3 ,.,.) = 
 -1.3863
 -1.3620
 -1.5277
 -1.5857
 -1.5879
 -1.3646
 -1.6233
 -1.3639
 -1.4901
 -1.6019
 -1.3473
 -1.6764
 -1.6356
 -1.3356
 -1.6660
 -1.3132

(4 ,.,.) = 
 -1.4003
 -1.3757
 -1.5431
 -1.6017
 -1.6040
 -1.3784
 -1.6397
 -1.3776
 -1.5051
 -1.6181
 -1.3609
 -1.6933
 -1.6521
 -1.3491
 -1.6828
 -1.3265

(5 ,.,.) = 
 -1.4144
 -1.3896
 -1

Updates 2000, num timesteps 160080, FPS 143, mean/median reward -20.4/-21.0, min/max reward -21.0/-19.0, entropy 1.79144, value loss 0.03326, policy loss -0.00303

 rollout returns 
(0 ,.,.) = 
 -1.6549
 -1.6591
 -2.3052
 -1.4295
 -1.3554
 -1.4636
 -1.7154
 -1.3191
 -1.3484
 -1.4270
 -1.3530
 -1.3425
 -1.3361
 -1.3900
 -1.4474
 -1.2709

(1 ,.,.) = 
 -1.6716
 -1.6758
 -2.3284
 -1.4440
 -1.3691
 -1.4784
 -1.7327
 -1.3324
 -1.3620
 -1.4414
 -1.3666
 -1.3560
 -1.3496
 -1.4040
 -1.4620
 -1.2838

(2 ,.,.) = 
 -1.6885
 -1.6928
 -2.3520
 -1.4586
 -1.3829
 -1.4933
 -1.7502
 -1.3458
 -1.3757
 -1.4560
 -1.3804
 -1.3697
 -1.3632
 -1.4182
 -1.4767
 -1.2967

(3 ,.,.) = 
 -1.7055
 -1.7099
 -1.3656
 -1.4733
 -1.3969
 -1.5084
 -1.7679
 -1.3594
 -1.3896
 -1.4707
 -1.3944
 -1.3836
 -1.3770
 -1.4325
 -1.4917
 -1.3098

(4 ,.,.) = 
 -1.7228
 -1.7271
 -1.3794
 -1.4882
 -1.4110
 -1.5236
 -1.7858
 -1.3732
 -1.4037
 -1.4855
 -1.4085
 -1.3976
 -1.3909
 -1.4470
 -1.5067
 -1.3231

(5 ,.,.) = 
 -1.7402
 -1.7446
 -1

...

Updates 16000, num timesteps 1280080, FPS 174, mean/median reward -18.7/-19.0, min/max reward -21.0/-16.0, entropy 1.78228, value loss 0.01164, policy loss -0.03224

 rollout returns 
(0 ,.,.) = 
 -1.3313
 -0.6801
 -0.8587
 -0.6370
 -1.9971
 -1.0527
 -1.1471
 -0.9045
 -0.7227
 -1.2611
 -1.0945
 -1.0444
 -1.5294
 -1.2208
 -0.9899
 -1.1472

(1 ,.,.) = 
 -1.3448
 -0.6869
 -0.8674
 -0.6434
 -2.0173
 -1.0633
 -1.1587
 -0.9136
 -0.7300
 -1.2739
 -1.1056
 -1.0549
 -1.5448
 -1.2332
 -0.9999
 -1.1588

(2 ,.,.) = 
 -1.3584
 -0.6939
 -0.8761
 -0.6499
 -2.0377
 -1.0741
 -1.1704
 -0.9229
 -0.7373
 -1.2867
 -1.1167
 -1.0656
 -1.5604
 -1.2456
 -1.0100
 -1.1705

(3 ,.,.) = 
 -1.3721
 -0.7009
 -0.8850
 -0.6565
 -1.0482
 -1.0849
 -1.1823
 -0.9322
 -0.7448
 -1.2997
 -1.1280
 -1.0764
 -1.5762
 -1.2582
 -1.0202
 -1.1823

(4 ,.,.) = 
 -1.3859
 -0.7080
 -0.8939
 -0.6631
 -1.0587
 -1.0959
 -1.1942
 -0.9416
 -0.7523
 -1.3128
 -1.1394
 -1.0872
 -1.5921
 -1.2709
 -1.0305
 -1.1943

(5 ,.,.) = 
 -1.3999
 -0.7151
 

Updates 16500, num timesteps 1320080, FPS 174, mean/median reward -18.6/-19.0, min/max reward -21.0/-17.0, entropy 1.76449, value loss 0.00636, policy loss -0.00413

 rollout returns 
(0 ,.,.) = 
 -1.0510
 -1.0041
 -1.1044
 -1.1147
 -1.3309
 -1.2836
 -1.4300
 -1.0842
 -0.9838
 -0.9522
 -0.4781
 -1.1026
 -1.1906
 -0.9820
 -1.8695
 -1.1839

(1 ,.,.) = 
 -1.0616
 -1.0142
 -1.1156
 -1.1259
 -1.3443
 -1.2965
 -1.4444
 -1.0952
 -0.9937
 -0.9618
 -0.4829
 -1.1137
 -1.2027
 -0.9919
 -1.8884
 -1.1959

(2 ,.,.) = 
 -1.0724
 -1.0245
 -1.1269
 -1.1373
 -1.3579
 -1.3096
 -1.4590
 -1.1063
 -1.0038
 -0.9715
 -0.4878
 -1.1249
 -1.2148
 -1.0020
 -1.9075
 -1.2080

(3 ,.,.) = 
 -1.0832
 -1.0348
 -1.1382
 -1.1488
 -1.3716
 -1.3229
 -1.4738
 -1.1174
 -1.0139
 -0.9814
 -0.4927
 -1.1363
 -1.2271
 -1.0121
 -0.9166
 -1.2202

(4 ,.,.) = 
 -1.0941
 -1.0453
 -1.1497
 -1.1604
 -1.3855
 -1.3362
 -1.4886
 -1.1287
 -1.0241
 -0.9913
 -0.4977
 -1.1478
 -1.2395
 -1.0223
 -0.9259
 -1.2325

(5 ,.,.) = 
 -1.1052
 -1.0558
 

Process Process-22:
Process Process-25:
Process Process-24:
Process Process-31:
Process Process-29:
Process Process-30:
Process Process-32:
Process Process-18:
Process Process-27:
Process Process-19:
Process Process-28:
Process Process-21:
Process Process-20:
Process Process-23:
Process Process-26:
Process Process-17:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):


  File "/usr/lib/python3.5/multiprocessing/connection.py", line 250, in recv
    buf = self._recv_bytes()
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 250, in recv
    buf = self._recv_bytes()
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
KeyboardInterrupt
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 250, in recv
    buf = self._recv_bytes()
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
KeyboardInterrupt
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 250, in recv
    buf = self._recv_bytes()
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._re

KeyboardInterrupt: 

In [8]:
from utils import orthogonal

class Categorical(nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super(Categorical, self).__init__()
        self.linear = nn.Linear(num_inputs, num_outputs)

    def forward(self, x):
        x = self.linear(x)
        return x

    def sample(self, x, deterministic):
        x = self(x)

        probs = F.softmax(x)
        if deterministic is False:
            action = probs.multinomial()
        else:
            action = probs.max(1)[1]
        return action

    def logprobs_and_entropy(self, x, actions):
        x = self(x)

        log_probs = F.log_softmax(x)
        probs = F.softmax(x)

        action_log_probs = log_probs.gather(1, actions)

        dist_entropy = -(log_probs * probs).sum(-1).mean()
        return action_log_probs, dist_entropy
    

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1 or classname.find('Linear') != -1:
        orthogonal(m.weight.data)
        if m.bias is not None:
            m.bias.data.fill_(0)


class CNNPolicy(nn.Module):
    def __init__(self, num_inputs, action_space):
        super(CNNPolicy, self).__init__()
        self.conv1 = nn.Conv2d(num_inputs, 32, 8, stride=4)
        self.conv2 = nn.Conv2d(32, 64, 4, stride=2)
        self.conv3 = nn.Conv2d(64, 32, 3, stride=1)

        self.linear1 = nn.Linear(32 * 7 * 7, 512)

        self.critic_linear = nn.Linear(512, 1)

        num_outputs = action_space.n
        self.dist = Categorical(512, num_outputs)

        self.train() # training mode. Only affects dropout, batchnorm etc
        self.reset_parameters()
        
    def act(self, inputs, states, masks, deterministic=False):
        value, x, states = self(inputs, states, masks)
        action = self.dist.sample(x, deterministic=deterministic)
        action_log_probs, dist_entropy = self.dist.logprobs_and_entropy(x, action)
        return value, action, action_log_probs, states

    def evaluate_actions(self, inputs, states, masks, actions):
        value, x, states = self(inputs, states, masks)
        action_log_probs, dist_entropy = self.dist.logprobs_and_entropy(x, actions)
        return value, action_log_probs, dist_entropy, states

    @property
    def state_size(self):
        return 1

    def reset_parameters(self):
        self.apply(weights_init)

        relu_gain = nn.init.calculate_gain('relu')
        self.conv1.weight.data.mul_(relu_gain)
        self.conv2.weight.data.mul_(relu_gain)
        self.conv3.weight.data.mul_(relu_gain)
        self.linear1.weight.data.mul_(relu_gain)

    def forward(self, inputs, states, masks):
        x = self.conv1(inputs / 255.0)
        x = F.relu(x)

        x = self.conv2(x)
        x = F.relu(x)

        x = self.conv3(x)
        x = F.relu(x)

        x = x.view(-1, 32 * 7 * 7)
        x = self.linear1(x)
        x = F.relu(x)

        return self.critic_linear(x), x, states

In [9]:
class RolloutStorage(object):
    def __init__(self, num_steps, num_processes, obs_shape, action_space, state_size):
        self.observations = torch.zeros(num_steps + 1, num_processes, *obs_shape)
        self.states = torch.zeros(num_steps + 1, num_processes, state_size)
        self.rewards = torch.zeros(num_steps, num_processes, 1)
        self.value_preds = torch.zeros(num_steps + 1, num_processes, 1)
        self.returns = torch.zeros(num_steps + 1, num_processes, 1)
        self.action_log_probs = torch.zeros(num_steps, num_processes, 1)
        
        action_shape = 1

        self.actions = torch.zeros(num_steps, num_processes, action_shape)
            
        self.actions = self.actions.long()
        self.masks = torch.ones(num_steps + 1, num_processes, 1)

    def cuda(self):
        self.observations = self.observations.cuda()
        self.states = self.states.cuda()
        self.rewards = self.rewards.cuda()
        self.value_preds = self.value_preds.cuda()
        self.returns = self.returns.cuda()
        self.action_log_probs = self.action_log_probs.cuda()
        self.actions = self.actions.cuda()
        self.masks = self.masks.cuda()

    def insert(self, step, current_obs, state, action, action_log_prob, value_pred, reward, mask):
        self.observations[step + 1].copy_(current_obs)
        self.states[step + 1].copy_(state)
        self.actions[step].copy_(action)
        self.action_log_probs[step].copy_(action_log_prob)
        self.value_preds[step].copy_(value_pred)
        self.rewards[step].copy_(reward)
        self.masks[step + 1].copy_(mask)

    def after_update(self):
        self.observations[0].copy_(self.observations[-1])
        self.states[0].copy_(self.states[-1])
        self.masks[0].copy_(self.masks[-1])

    def compute_returns(self, next_value, use_gae, gamma, tau):
        self.returns[-1] = next_value
        for step in reversed(range(self.rewards.size(0))):
            self.returns[step] = self.returns[step + 1] * \
                gamma * self.masks[step + 1] + self.rewards[step]


In [10]:
from multiprocessing import Process, Pipe

def worker(remote, parent_remote, env_fn_wrapper):
    parent_remote.close()
    env = env_fn_wrapper.x()
    while True:
        cmd, data = remote.recv()
        if cmd == 'step':
            ob, reward, done, info = env.step(data)
            if done:
                ob = env.reset()
            remote.send((ob, reward, done, info))
        elif cmd == 'reset':
            ob = env.reset()
            remote.send(ob)
        elif cmd == 'reset_task':
            ob = env.reset_task()
            remote.send(ob)
        elif cmd == 'close':
            remote.close()
            break
        elif cmd == 'get_spaces':
            remote.send((env.action_space, env.observation_space))
        else:
            raise NotImplementedError


class CloudpickleWrapper(object):
    """
    Uses cloudpickle to serialize contents (otherwise multiprocessing tries to use pickle)
    """
    def __init__(self, x):
        self.x = x
    def __getstate__(self):
        import cloudpickle
        return cloudpickle.dumps(self.x)
    def __setstate__(self, ob):
        import pickle
        self.x = pickle.loads(ob)



class SubprocVecEnv(object):
    def __init__(self, env_fns):
        """
        envs: list of gym environments to run in subprocesses
        """
        self.closed = False
        nenvs = len(env_fns)
        self.remotes, self.work_remotes = zip(*[Pipe() for _ in range(nenvs)])
        self.ps = [Process(target=worker, args=(work_remote, remote, CloudpickleWrapper(env_fn)))
            for (work_remote, remote, env_fn) in zip(self.work_remotes, self.remotes, env_fns)]
        for p in self.ps:
            p.daemon = True # if the main process crashes, we should not cause things to hang
            p.start()
        for remote in self.work_remotes:
            remote.close()

        self.remotes[0].send(('get_spaces', None))
        self.action_space, self.observation_space = self.remotes[0].recv()


    def step(self, actions):
        for remote, action in zip(self.remotes, actions):
            remote.send(('step', action))
        results = [remote.recv() for remote in self.remotes]
        obs, rews, dones, infos = zip(*results)
        return np.stack(obs), np.stack(rews), np.stack(dones), infos

    def reset(self):
        for remote in self.remotes:
            remote.send(('reset', None))
        return np.stack([remote.recv() for remote in self.remotes])

    def reset_task(self):
        for remote in self.remotes:
            remote.send(('reset_task', None))
        return np.stack([remote.recv() for remote in self.remotes])

    def close(self):
        if self.closed:
            return

        for remote in self.remotes:
            remote.send(('close', None))
        for p in self.ps:
            p.join()
        self.closed = True

    @property
    def num_envs(self):
        return len(self.remotes)

